<a href="https://colab.research.google.com/github/Jepees/StepAhead-Nuevo/blob/main/scripts/3_labeling_sentimen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import openai
import pandas as pd
import time
import tiktoken
import os

xxx = "" #token API OpenAI

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Jepees/StepAhead-Nuevo/refs/heads/main/data/data_clean_merge.csv")
df

,Brand,nama barang,kategori,harga,harga_int,user,varian,ukuran,date,review,rating,terbantu
0,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,A***s,NaN,NaN,July 2025,Sepatunya ok dan pas dengan ukuran.. Nyampenya...,5,0
1,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,Z***i,NaN,NaN,July 2025,barang original enak di pakek ngebut sepatunya,5,0
2,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,R***i,NaN,NaN,July 2025,Bagus nyaman melebihi ekspektasi saya,5,0
3,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,V***u,NaN,NaN,July 2025,Produknya mantapppplll,5,1
4,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,Y***e,NaN,NaN,July 2025,Pengiriman cepat,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
13908,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,A***e,NaN,38.0,April 2024,NaN,4,0
13909,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,D***i,NaN,40.0,April 2024,NaN,5,0
13910,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,d***a,NaN,42.0,April 2024,Lumyan,5,0
13911,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,r***n,NaN,41.0,April 2024,NaN,3,0


## 1st Labelling

In [ ]:
openai.api_key = xxx

def classify_sentiment(text):
    try:
        response = openai.ChatCompletion.create(
            # model="gpt-4o-mini",
            model="gpt-4.1-mini",
            # model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": (
                    "Anda adalah asisten analisis sentimen untuk ulasan produk sepatu dari marketplace online.\n\n"
                    "Tugas Anda adalah membaca ulasan pengguna terhadap produk sepatu dari brand seperti 910, Ortuseight, Mills, atau Nuevo, "
                    "kemudian mengklasifikasikan sentimen terhadap produk tersebut sebagai:\n"
                    "- 1 jika ulasan hanya berisi pujian atau komentar positif **tanpa menyebutkan kekurangan sama sekali**.\n"
                    "- -1 jika ulasan mengandung **kritik sekecil apa pun**, termasuk menyebut cacat produk, kekurangan, keluhan ringan, atau perbandingan negatif.\n"
                    "- 0 jika ulasan bersifat ambigu, netral, atau hanya menjelaskan fitur produk tanpa opini jelas.\n\n"
                    "Contoh:\n"
                    "- \"Cuma lemnya aja gak rapi blepotan. Secara keseluruhan bagus\" → -1 (karena menyebutkan kekurangan)\n"
                    "- \"Sesuai gambar, oke banget!\" → 1\n"
                    "- \"Datang cepat, belum dicoba\" → 0\n\n"
                    "Jawaban Anda harus hanya berupa satu angka: -1, 0, atau 1."
                )},
                {"role": "user", "content": f"Teks ulasan: {text}"}
            ],
            temperature=0
        )

        label_text = response['choices'][0]['message']['content'].strip()
        if label_text in ['-1', '0', '1']:
            return int(label_text)
        else:
            print("Output tidak sesuai:", label_text)
            return None

    except Exception as e:
        print("Error:", e)
        return None


In [ ]:
teks = df['review'].loc[146]
print(teks)
classify_sentiment(teks)

Untuk produk sih bagus...cuman agak kecewa sm pengiriman nya 1 mgg lbh, pdhl biasanya 3/4 hari sudah sampai..


-1

In [ ]:
dop = df['review'].loc[140:160]
for i in dop.index:
    print(f"{i}: {dop.loc[i]}")

140: produk sesuai deskripsi. tp saya kekecilan sizenya  padahal insole udah sama. untung cukup di kaki anak saya. thanks 910
141: Sol nya pas tp polanya gak bener
142: Ukuran harus lebih 1 size. Aku biasa pake 40 ini kekecilan, tukar deh jadi 41
143: Sepatu kekecilan sebaiknya klo biasanya ukuran 38 beli yg lebih besar lg spy ga kesempitan
144: Sedih kekecilannnnnn bgus kokkkk
145: Barangnya telah sampai sesuai dengan pesanan.... Sepatunya ringan sekali.... Pokoknya bagus sekali.. Rekomendasi beli lagi de
146: Untuk produk sih bagus...cuman agak kecewa sm pengiriman nya 1 mgg lbh, pdhl biasanya 3/4 hari sudah sampai..
147: Sepatunya bagus, emang btl ya, size yg biasa di pakai kekecilan. Tks
148: Cute banget warnanya licuuuuuj
149: mantap, keren, dan tidak mengecewakan pokoknya👍🏻
150: nyaman dipakai
151: received ✅
152: Suka banget aayo pada beli
153: Kren mantul😍
154: Keren
155: tq
156: ..
157: Bentuk dan ukuran: Pas sesuai pesanan ,pengiriman cepat ,pokoknya best lah ,sukses terus bu

In [ ]:
# batching data
df1 = df.loc[:3000]
df2 = df.loc[3001:6000]
df3 = df.loc[6001:9000]
df4 = df.loc[9001:12000]
df5 = df.loc[12001:]

### df1

In [ ]:
# Tambahkan kolom label_sentimen
df1['label_sentimen'] = df1['review'].apply(lambda x: classify_sentiment(x))
df1.to_csv("df1.csv")

C:\Users\Dell\AppData\Local\Temp\ipykernel_14064\1439696164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['label_sentimen'] = df1['review'].apply(lambda x: classify_sentiment(x))


### df2

In [ ]:
# Tambahkan kolom label_sentimen
df2['label_sentimen'] = df2['review'].apply(lambda x: classify_sentiment(x))
df2.to_csv("df2.csv")

Output tidak sesuai: Silakan berikan teks ulasan yang ingin Anda analisis, dan saya akan membantu mengklasifikasikan sentimennya.


C:\Users\Dell\AppData\Local\Temp\ipykernel_14064\3363900631.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['label_sentimen'] = df2['review'].apply(lambda x: classify_sentiment(x))


### df3

In [ ]:
# Tambahkan kolom label_sentimen
df3['label_sentimen'] = df3['review'].apply(lambda x: classify_sentiment(x))
df3.to_csv("df3.csv")

Error: HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
)
Output tidak sesuai: Silakan berikan teks ulasan yang ingin Anda analisis, dan saya akan memberikan klasifikasi sentimennya.


C:\Users\Dell\AppData\Local\Temp\ipykernel_14064\1553068472.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['label_sentimen'] = df3['review'].apply(lambda x: classify_sentiment(x))


### df4

In [ ]:
# Tambahkan kolom label_sentimen
df4['label_sentimen'] = df4['review'].apply(lambda x: classify_sentiment(x))
df4.to_csv("df4.csv")

C:\Users\Dell\AppData\Local\Temp\ipykernel_10824\1020095231.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['label_sentimen'] = df4['review'].apply(lambda x: classify_sentiment(x))


### df5

In [ ]:
# Tambahkan kolom label_sentimen
df5['label_sentimen'] = df5['review'].apply(lambda x: classify_sentiment(x))
df5.to_csv("df5.csv")

Error: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
Output tidak sesuai: Ulasan ini tidak memberikan informasi yang cukup untuk diklasifikasikan. Mohon berikan ulasan yang lebih lengkap. Terima kasih!
Output tidak sesuai: Ulasan ini tidak memberikan informasi yang cukup untuk diklasifikasikan. Silakan berikan ulasan yang lebih lengkap. Terima kasih!
Output tidak sesuai: Maaf, tapi saya memerlukan teks ulasan mengenai produk sepatu agar dapat memberikan klasifikasi sentimen. Silakan berikan ulasan mengenai produk sepatu tersebut. Terima kasih!


C:\Users\Dell\AppData\Local\Temp\ipykernel_10824\2645783086.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['label_sentimen'] = df5['review'].apply(lambda x: classify_sentiment(x))


## Merging

In [ ]:
# datas = {f'df{i+1}': pd.read_csv(f'./folder labeling/1st labeling/{os.listdir("./folder labeling/1st labeling")[i]}', index_col='Unnamed: 0') for i in range(len(os.listdir("./folder labeling/1st labeling")))}
df = pd.concat([df1,df2,df3,df4,df5])
df.to_csv("data_with_sentimen.csv", index=False)
df

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Jepees/StepAhead-Nuevo/refs/heads/main/data/data_with_sentimen.csv')
df

,Brand,nama barang,kategori,harga,harga_int,user,varian,ukuran,date,review,rating,terbantu,label_sentimen
0,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,A***s,NaN,NaN,July 2025,Sepatunya ok dan pas dengan ukuran.. Nyampenya...,5,0,1
1,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,Z***i,NaN,NaN,July 2025,barang original enak di pakek ngebut sepatunya,5,0,1
2,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,R***i,NaN,NaN,July 2025,Bagus nyaman melebihi ekspektasi saya,5,0,1
3,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,V***u,NaN,NaN,July 2025,Produknya mantapppplll,5,1,1
4,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,Y***e,NaN,NaN,July 2025,Pengiriman cepat,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13910,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,A***e,NaN,38.0,April 2024,NaN,4,0,0
13911,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,D***i,NaN,40.0,April 2024,NaN,5,0,0
13912,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,d***a,NaN,42.0,April 2024,Lumyan,5,0,0
13913,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,r***n,NaN,41.0,April 2024,NaN,3,0,0


In [ ]:
df['label_sentimen'].value_counts()

label_sentimen
 1    9121
 0    2814
-1    1980
Name: count, dtype: int64

In [ ]:
for j in [-1, 0, 1]:
    x = df[df['label_sentimen'] == j]['review']
    print(f'untuk sentimen {j}:')
    for i in range(100, 120):
        print(f'{i}: {x.iloc[i]}')
    print('\n')

untuk sentimen -1:
100: overall sepatunya bagus, dari bahan, warna, dan Kualitas, cuman mungkin stock terakhir jadi ada beberapa noda di midsolenya, Terimakasih Nineteen, bangga Produk Lokal
101: KUALITAS BAIK , TAPI 46 NYA BEDA DENGAN 46 MERK SCHE**RS BUT OVERALL OKE Modelnya keren!.
102: respon cepat cuma masih keras  mudah2an lama2 bisa lebih empuk
103: the besttt. buat harga segini udh oke banget fitur2nya jgn minta aneh2 dibawah 500rb cuma packingnya aja agak menyedihkan pake bubble warp doang jadi box pasti ancur kalo... 
104: Kwalitas ok, hanya saja tiba2 hrs upsize 1 nmr utk produk yg tersedia nya
105: Seller (lumayan fast respon dan enak waktu tanya-tanya barangnya), Barang (Original yang pasti, local pride, keren parah), Packing (Kurang aman karena pakai bubble wrap t... 
106: pertama kali mencoba lari 5K, masih terasa kurang nyaman..mungkin harus dicoba beberapa kali baru sesuai dengan bentuk kaki..dari sisi design sepatu ini sudah sangat bagu... 
107: sepatu 910 kedua yg sy